In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import re
from urllib.parse import urlparse
from pandas import read_excel


In [6]:
!pip install openpyxl

In [31]:
# data = pd.DataFrame({
#     'URLs': ['http://example.com/login','https://example.com/dasf', 'https://secure.bank.xyz', 'https://login.fakebank.com','http://login.fakebank.com'],
#     'Fraud': [0, 1, 1, 1, 0]  # 0 = Not Fraudulent, 1 = Fraudulent
# })
data=pd.read_csv('/content/excel-bank-new.csv')


In [32]:
def extract_features(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    # Extract features
    features = {
        'url_length': len(url),
        'dot_count': url.count('.'),
        'special_char_count': url.count('@') + url.count('-') + url.count('/'),
        'contains_secure': int('secure' in url),
        'contains_login': int('login' in url),
        'is_https': int(parsed_url.scheme == 'https'),
        'is_suspicious_https': int(is_suspicious_https(url))  # Check for suspicious https usage
    }
    features['domain_length'] = len(domain)

    return features

In [33]:
def is_suspicious_https(url):
    parsed_url = urlparse(url)
    # Check if it starts with https but contains suspicious keywords or patterns
    if parsed_url.scheme == 'https':
        # Look for suspicious subdomains or phishing-related keywords
        suspicious_keywords = ['login', 'secure', 'verify', 'account']
        if any(keyword in parsed_url.path for keyword in suspicious_keywords):
            return True
    return False

In [34]:
data.columns
data['URL'].dropna()
data.isnull()

,URL,Fraud
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
466,True,True
467,True,True
468,True,True
469,True,True


In [35]:
print(data['URL'].dtype)
print(data['URL'].head())

object
0                       https://www.onlinesbi.sbi/
1                         http//www.onlinesbi.sbi/
2    https://retail.onlinesbi.sbi/retail/login.htm
3     htts://retail.onlinesbi.sbi/retail/login.htm
4                                https://bank.sbi/
Name: URL, dtype: object


In [51]:

data = data.dropna()



In [52]:
data['URL'] = data['URL'].fillna('').astype(str)
features = pd.DataFrame([extract_features(url) for url in data['URL']])

# Add the features to the original dataset
data = pd.concat([data, features], axis=1)

# Define your features (X) and target (y)
X = data.drop(columns=['URL', 'Fraud'])  # Drop URLs and target column
y = data['Fraud']








In [53]:

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent'
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


# Standardize the features (important for some ML models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

y_test.isnull().sum()



0

In [55]:
# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        43

    accuracy                           1.00        86
   macro avg       1.00      1.00      1.00        86
weighted avg       1.00      1.00      1.00        86

